Appologies to the reader: there is a lot of code in this post, but if you stay til the end there is a cookie emoji for you! ➗ The City of Cape Town have a great initialive for [Open Data](http://web1.capetown.gov.za/web1/opendataportal/default), with huge volumns of data of rental price, wifi usage and service delivery requests. 💽 This data is extensive, but has- as extected- many challenges to visualising and understanding the data itself.  📊
  
Let's import some libraries!📚

In [19]:
import os
import string
from multiprocessing import pool, cpu_count

import numpy as np
import pandas as pd
import scipy as sp

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.utils.validation import check_symmetric
from sklearn.base import TransformerMixin
from sklearn.cluster import FeatureAgglomeration, AffinityPropagation, DBSCAN
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.manifold import TSNE,smacof, MDS
from sklearn.decomposition import PCA
from sklearn.feature_extraction import stop_words
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelBinarizer


from gensim.models.deprecated.old_saveload import SaveLoad
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py

The data has some weird formatting, and a number of the files are either corrupt or formatted as xls and xlsx file types. 📁 Any-who, I import a ransom rample of them to play around with a see what I could find.  😕

In [2]:
parent = os.path.join('Data','CapeTownServiceCalls')
data = pd.DataFrame(None)

for file in os.listdir(parent):
    if file.endswith(".xlsx") & (os.path.getsize(os.path.join(parent, file))!=0):
        child = os.path.join(parent, file)
        load = pd.read_excel(child, na_values=['Not assigned','','nan','#'])
        load.columns = load.iloc[3,:]
        load = load.drop([0,1,2,3], axis=0).dropna()
        load['file'] = file
        
        data = pd.concat([data,load], axis=0, sort=False)
        print(file)

Service requests September 2017.xlsx
Service requests October 2017.xlsx
Service requests August 2017.xlsx
Service requests December 2017.xlsx
Service requests August 2016.xlsx
Service requests July 2017.xlsx
Service requests November 2017.xlsx


The first thing you will notice in the dataset, is that the data is insane! 😮 It's medium-size, but has no clear standardization. 🐘 Fields like description and suburb allow users to fill in whatever they want and users are terrible at spelling! ✅ The data has this strange formatting for coordinated, which isn't in normal longitude and lattitude, but is also either massively positive or massively negative and has some unknown origin... This I think was some annonymization so I didn't too hard to try deanonymize it on a public blog.  🗺

In [3]:
columns = data.columns.tolist()
columns[6] = 'Description'
data.columns = columns

data.shape

(461301, 12)

In [4]:
data.sample(n=7)

,Sub Council,Ward,Suburb,C3 Complaint Type,Work Center,Notification,Description,X-Y Co ordinate 1,X-Y Co ordinate 2,Created On Date,Notifications Created,file
32904,Sub-Council 2,102,VREDEKLOOF,Internal : Billing / Adjustment Queries,Scottsdene: Water,1012539430,Internal : Billing / Adjustment Queries,30142.140000,3748999.620000,2017-08-02 00:00:00,1,Service requests August 2017.xlsx
23561,Sub-Council 17,048,BELTHORN ESTATE,4B: Investigation,Water Restrictions 4B CAPE TOWN,1012989580,4B Contravention,44607.960000,3761880.400000,2017-12-11 00:00:00,1,Service requests December 2017.xlsx
58902,Sub-Council 5,031,BONTEHEUWEL,Carpentry,Housing - Bishop Lavis: Carpentry,1012738396,Lounge window frame and handle loose,-40424.6883337413,-3759202.43666664,2017-10-03 00:00:00,1,Service requests October 2017.xlsx
25076,Sub-Council 16,077,GARDENS,PPM Customer Unable to Purchase,Electricity Generation & Distribution,1012864900,1012864900,54845.080000,3756384.310000,2017-11-06 00:00:00,1,Service requests November 2017.xlsx
67385,Sub-Council 8,086,LWANDLE,PPM Faulty (Enter detailed description),Electricity-First Line Response_Helderbe,1011405110,PPM Faulty 21798 VULINDLELA ROAD GIFT,-12394.6996,-3777127.1098,2016-08-14 00:00:00,1,Service requests August 2016.xlsx
30915,Sub-Council 19,067,SEAWINDS,No Water WMD,Cape Town / SPM,1011413323,No Water WMD,-47649.9665,-3772489.5394,2016-08-16 00:00:00,1,Service requests August 2016.xlsx
70665,Sub-Council 7,103,BONNIE BROOK,High Water Pressure,Scottsdene: Water,1012942094,High Water Pressure,27816.280000,3745484.380000,2017-11-27 00:00:00,1,Service requests November 2017.xlsx


In [5]:
data.nunique()

Sub Council                  26
Ward                        217
Suburb                     5108
C3 Complaint Type           557
Work Center                 979
Notification             461301
Description              159969
X-Y Co ordinate 1        296441
X-Y Co ordinate 2        294635
Created On Date             214
Notifications Created         2
file                          7
dtype: int64

___

# First Problem: Suburb Descriptions
As it turns out, they do not have a drop-down for suburb descriptions, so people just put down what they want and make a lot of spelling mistakes. 🎫 There are only a hand-full on suburbs in Cape Town but there are over 3300 labels, most of which are tiny one-offs.  🗳

In [6]:
data.Suburb = data.Suburb.str.lower().str.replace('[^a-z\s]', ' ').str.split().str.join(' ').str.lstrip(' ').str.rstrip(' ')
labels = data.Suburb.value_counts()
labels.shape

(3359,)

A classic method to calculate the distance between words or phrases in some spellcheck is using the Levenshtein Distance, but this is a computationally intensive distance metric, which despite my best efforts parallelizing the code, is stupidly slow to compute. 🤖 So what we are going to try is to compute the tdidf of the charcaters, then calculate the cosine similarity of the phrases.  Because there are so many entries, we are going to try cheating and rather than computing the distance for all points, we are just going to do unique points, and make a adjustmnets in order to approximate from density based method, by adjusting for the number of entries with that label, as well and the length of the descriptions. 🗜  This is much much much faster to compute, so I'm going to be as lazy as possible here! 🏇

In [7]:
tfidf = TfidfVectorizer(analyzer='char')

char_vectors = tfidf.fit_transform(labels.index)
char_distances = pairwise_distances(char_vectors, metric='euclidean', n_jobs=-1)

size_regulaization = pd.Series(labels.values)
size_regulaization = np.log(size_regulaization)
size_regulaization = (size_regulaization/(size_regulaization.var()**1.5))+1

size_weighted_distances = pd.DataFrame(np.transpose(char_distances/size_regulaization.values.reshape(-1,1)), columns=labels.index, index=labels.index)

length = pd.Series(labels.index).apply(lambda x: len(x))
length_regularize = length.values.reshape(-1,1)+(np.transpose(length.values.reshape(-1,1)))
length_regularize = length_regularize - length_regularize.mean()
length_regularize = length_regularize/(length_regularize.var())
length_regularize = length_regularize + 1

#Arithmetic Mean
#length_regularize = length.values.reshape(-1,1).dot(np.transpose(length.values.reshape(-1,1)))**0.5

sizelength_weighted_distances = pd.DataFrame(size_weighted_distances/length_regularize)
sizelength_weighted_distances.values.shape

(3359, 3359)

In [8]:
mds = TSNE(n_components=2, metric='precomputed')
sizelength_weighted_distances = check_symmetric(sizelength_weighted_distances.astype(np.float64))
mds_transform = mds.fit_transform(sizelength_weighted_distances.astype(np.float64).values)
mds_transform = pd.DataFrame(mds_transform, columns=['x','y'])
word_clustering = FeatureAgglomeration(n_clusters=2000, affinity='precomputed', memory=None, connectivity=None, compute_full_tree=True, linkage='complete')
clusters = pd.DataFrame(word_clustering.fit_transform(-sizelength_weighted_distances))
clusters = clusters.idxmax(1)
clusters.index = labels.index

/home/marcussky/.conda/envs/tulip/lib/python3.6/site-packages/sklearn/utils/validation.py:709: UserWarning:

Array is not symmetric, and will be converted to symmetric by average with its transpose.



So lets try visualize these weight spellings and groups we can form from them. 🖥 I know this looks hard to visualize here, but the main here is not to do clustering, but spell-check so we want to be as gentle as posssible.  I make the size indicate the size of the surburb tag in the corpus.  💯

In [9]:
init_notebook_mode(connected=True)
py.iplot([go.Scatter(x=mds_transform.x.tolist(), y=mds_transform.y.tolist(),  mode = 'markers', text=(clusters.apply(lambda x: str(x) + ': ') + labels.index).values.tolist(), 
                 marker=dict(
                     size=(size_regulaization**4+3).tolist(),
                     color=clusters.tolist(),
                     colorscale='Viridis',
                 ))])

In [10]:
pd.Series(clusters.value_counts()).plot.hist()

Here at least for the big clusters, we can see it has worked pretty well.  Not perfect, I admit, but pretty well considering how quick it was.  

In [11]:
clusters.loc[clusters == clusters.value_counts().idxmax()]

khayelitsha           1383
khayelisha            1383
khayalitsha           1383
khayalitcha           1383
khayelitsa            1383
kahyelitsha           1383
khayelithsa           1383
khaylitsha            1383
khayeltsha            1383
siyahlala             1383
khayekitsha           1383
khkayelistha          1383
khaylistha            1383
kayelitsha            1383
highlay               1383
khayaletisha          1383
kihayelitsha          1383
khayalitha            1383
eyethu khayelitsha    1383
khayelistha           1383
dtype: int64

In [12]:
clusters = pd.concat([clusters, labels], axis=1, sort=False)
clusters.columns = ['cluster', 'count']
dictionary = pd.merge(clusters,clusters.groupby('cluster').idxmax(), how='left', on='cluster')
dictionary['old_name'] = clusters.index
dictionary.index = clusters.index
dictionary.columns = ['cluster','count','new_name','old_name']

data = pd.merge(data, dictionary, how='left', left_on='Suburb', right_on='old_name')

Look this could also be improved by doing clustering on words, then on sentences, but the issue comes in words which are randomly split.  So at somepoints you are going to have to do this accross the whole phrase, you must just decide when.  

____

# Second Problem: Descriptions! 

There are also a lot of descriptions, just a lot, as we want to check out themes these is a challenge as we need to figure out how to gorup them.  Now, a common technique for handling text and topics is a technique called Latent Drichlet Allocation- but I tried it and it wasn't the best at all.  This is for a couple of reasons 🔢:  
1.  The descriptions are really short, so don't have enough words to make it stable.  
2.  It would require me hand labelling the groupings and I'm very lazy when it comes to labelling.  
  
So how do we cheat.  This is the plan:  
0️⃣ Clean the text, remove the stop_words like 'the'  
1️⃣ Use word2vec to get word vectors  
2️⃣ Take a weighted average of the word vectors to get document vectors  
3️⃣ Use those to do clustering on topics or concepts  
4️⃣ Take a weighted average of the clusters to get a topics vector  
5️⃣ Use that topic vector to get a word vector nearest to it  
6️⃣ Visualize the whole thing for your lovely eyes to see  
  
Ready....Go! 🏌

In [13]:
data = data.reset_index()

In [14]:
vc = data.loc[:,'C3 Complaint Type'].value_counts()
vc.head(n=10)

Sewer: Blocked/Overflow                 68386
No Power                                32447
No Water WMD                            31310
Customer : Reconnection Request         16010
4B: Investigation                       11816
Leak at Water Meter / Stopcock          11323
WMD: Meter: Aqualoc Fault               10956
Customer : Meter Reading/Consumption     9303
No Water Supply                          8236
Stolen Bins - 240L                       8103
Name: C3 Complaint Type, dtype: int64

In [15]:
len(vc.index.tolist())

557

In [16]:
def RemoveStops(x, stops=None):
    """Removes stopwords from sentences
    
    Libraries & Versions:
    Python==3.6.5
    Pandas=='0.23.1' as pd
    nltk=='3.3'
    
    Keyword arguments:
    X -- Pandas Series of text as strings
    stops -- own stopwords provided
    """
    stop = list(stop_words.ENGLISH_STOP_WORDS) + ['may',
                                                  'also','zero','one','two',
                                                  'three','four','five','six','seven',
                                                  'eight','nine','ten','eleven','twelve',
                                                  'thirteen','fourteen','fifteen',
                                                  'sixteen','seventeen','eighteen','nineteen',
                                                  'twenty','thiry','forty','fifty','sixty',
                                                  'seventy','eighty','ninety','hundred',
                                                  'thousand','million','billion','trillion',
                                                  'accross','amoung','among','beside','within',
                                                  'company', 'holdings','inc','incorperated',
                                                  'group','limited','plc', 'companies',
                                                  'listings', 'list','of', 'stock',
                                                  'exchange']+list(string.ascii_lowercase)
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    x = x.str.replace(pat, '')
    x = x.str.replace(r'\s+', ' ')
           
    return x

In [17]:
class WeightedWordVectors(TransformerMixin):
    """Trains Collation Model to join co-occuring works into single entities or concepts
    
    Libraries & Versions:
    Python==3.6.5
    Pandas=='0.23.1' as pd
    nltk=='3.3'
    numpy=='1.14.5'
    
    Keyword arguments:
    X -- Pandas Series of text as strings
    """
    def __init__(self, model=None):
        pass
    
    def fit(self, X, y=None):
        # Train word vectors
        self.word2vec = Word2Vec(sentences=X.str.split(' '), min_count=1, size=50, workers=cpu_count()-3, iter=2)
        return self

    def transform(self, X):
        #Fit TFIDF
        self.tfidf = TfidfVectorizer(vocabulary= list(self.word2vec.wv.vocab.keys()))
        idfs = self.tfidf.fit_transform(X)
        inverse_idfs = self.tfidf.inverse_transform(idfs)
        
        #Weighted Vectors
        weighted_docs = []        
        for idf, inv in zip(idfs, inverse_idfs):
            try:
                weight = idf[idf!=0]
                vector = self.word2vec[inv]
                weighted_doc = weight.dot(vector)
            except:
                weighted_doc = np.empty((1, self.word2vec.wv.vector_size), float)    
            weighted_docs.append(weighted_doc.tolist()[0])
                
        return pd.DataFrame(weighted_docs).fillna(0)

In [20]:
pipe = make_pipeline(
    FunctionTransformer(func=lambda x: x.str.lower(), validate=False),
    FunctionTransformer(func=lambda x: x.str.replace('[^a-zA-Z]', ' '), validate=False),
    FunctionTransformer(func=RemoveStops, validate=False),
    WeightedWordVectors()
)

Description_Vectors = pipe.fit_transform(data.loc[:,'C3 Complaint Type'])
unique_descriptions = vc.index.str.lower().str.replace('[^a-z\s]', ' ').str.split().str.join(' ').str.lstrip(' ').str.rstrip(' ')
uniques_descriptions = pd.Series(unique_descriptions).loc[unique_descriptions!='']
Unique_Description_Vectors = pipe.transform(unique_descriptions)
Unique_Description_Vectors.shape

/home/marcussky/.conda/envs/tulip/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



(557, 50)

In [21]:
discription_tsne = TSNE(n_components=2)
dist_comp = discription_tsne.fit_transform(Unique_Description_Vectors)
dist_comp = pd.DataFrame(dist_comp, columns=['x','y'])

clustering_pipeline = make_pipeline(
    FunctionTransformer(lambda x: pairwise_distances(x, metric='cosine')),
    AffinityPropagation(damping=0.5, max_iter=1000, convergence_iter=400,affinity='precomputed', preference=vc.values/(547)),
)
discription_clusters = pd.Series(clustering_pipeline.fit_predict(dist_comp))
discription_clusters.nunique()

87

In [22]:
init_notebook_mode(connected=True)
py.iplot([go.Scatter(x=dist_comp.x.tolist(), y=dist_comp.y.tolist(),  mode = 'markers', text=(discription_clusters.apply(lambda x: str(x) + ': ') + uniques_descriptions).tolist(), 
                 marker=dict(
                     size=vc.apply(lambda x: np.log(x)*2).tolist(),
                     color=discription_clusters.tolist(),
                     colorscale='Viridis',
                 ))])

In [23]:
discription_clusters = pd.concat([discription_clusters, pd.Series(vc.index.values), pd.Series(vc.values)], axis=1, sort=False)
discription_clusters.columns = ['cluster','name', 'count']
description_dictionary = pd.merge(discription_clusters,discription_clusters.groupby('cluster').max(), how='left', on='cluster')
description_dictionary.columns = ['cluster','old_description','count_x','new_description','count_y']

cluster_word = pd.DataFrame()
for i in range(description_dictionary.cluster.nunique()):
    cluster_mean = pd.DataFrame()
    cluster_size = 0
    for i in description_dictionary.loc[description_dictionary.cluster == i,:].index:
        cluster_mean = pd.concat([cluster_mean, pd.DataFrame(Unique_Description_Vectors.loc[i,:] * (description_dictionary.loc[i,'count_x'])).transpose()], axis=0)
        cluster_size = cluster_size + (description_dictionary.loc[i,'count_x'])
    cluster_word = pd.concat([cluster_word,pd.DataFrame(cluster_mean.sum()/cluster_size).transpose()], axis=0)

cluster_word = cluster_word.reset_index()
cluster_word = cluster_word.drop(columns='index')

words = []
for word in range(0, cluster_word.shape[0]):
    words.append(pipe.named_steps.weightedwordvectors.word2vec.most_similar(positive=[cluster_word.iloc[word,:].values], topn=1)[0])
words = pd.DataFrame(words, columns=['word','similarity']).reset_index()

description_dictionary = pd.merge(description_dictionary, words, how='left', left_on='cluster', right_on='index')

data = pd.merge(data, description_dictionary, how='left', left_on='C3 Complaint Type', right_on='old_description')

/home/marcussky/.conda/envs/tulip/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning:

Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).



___

# Putting it together: Sankey Plot

So I recently discovered one of the coolest plots and [visualization of soy production is South Africa](https://trase.earth/flows).  Check it out! 🛎 They use a plot called a sankey plot which shows flows or relationships between equal sets.  Being lazy, I thought why dont I just get these relationships using a one-hot encoding and then OLS regression then times the coefficients but the size of the data to get weighted edges.  📏  It was a great hack and I recommend it to those wanting to enjoy more Sankey Plots in their lives! 🖼

In [24]:
largest_complaints = data.loc[:,'word'].value_counts().nlargest(n=10)
largest_suburbs = data.loc[:,'new_name'].value_counts().nlargest(n=5)
largest_centres = data.loc[:,'Work Center'].value_counts().nlargest(n=5)

sankey_sample = data.loc[(data.loc[:,'word'].isin(largest_complaints.index.tolist()))&
                          (data.loc[:,'new_name'].isin(largest_suburbs.index.tolist()))&
                          (data.loc[:,'Work Center'].isin(largest_centres.index.tolist())),:]

sparse_complaint = pd.get_dummies(sankey_sample.loc[:,'word'])
sparse_suburb = pd.get_dummies(sankey_sample.new_name)
sparse_centres = pd.get_dummies(sankey_sample.loc[:,'Work Center'])

lr = LinearRegression(fit_intercept=False)
lr.fit(sparse_complaint, sparse_suburb)
connections = lr.coef_*sparse_suburb.sum().values.reshape(-1,1).dot(np.transpose(sparse_complaint.sum().values.reshape(-1,1)))/sparse_suburb.sum().sum()

lr2 = LinearRegression(fit_intercept=False)
lr2.fit(sparse_centres, sparse_complaint)
connections2 = lr2.coef_*sparse_complaint.sum().values.reshape(-1,1).dot(np.transpose(sparse_centres.sum().values.reshape(-1,1)))/sparse_complaint.sum().sum()

In [25]:


sankey_data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = sparse_suburb.columns.tolist()+sparse_complaint.columns.tolist()+sparse_centres.columns.tolist()
    ),
    link = dict(
      source = lr.coef_.nonzero()[0].tolist()+[i+sparse_suburb.shape[1] for i in lr2.coef_.nonzero()[0].tolist()],
      target = [i+sparse_suburb.shape[1] for i in lr.coef_.nonzero()[1].tolist()]+[i+sparse_suburb.shape[1]+sparse_complaint.shape[1] for i in lr2.coef_.nonzero()[1].tolist()],
      value = connections[lr.coef_.nonzero()[0].tolist(),lr.coef_.nonzero()[1].tolist()].tolist()+connections2[lr2.coef_.nonzero()[0].tolist(),lr2.coef_.nonzero()[1].tolist()].tolist()
  ))

layout =  dict(
    title = "Service Delivery Flow",
    font = dict(
      size = 10
    )
)

fig = dict(data=[sankey_data], layout=layout)
py.iplot(fig, validate=False)

Here blocked is got to do with issues with public toilets from the data.  One series issue seems to be water, power, sanitation and theft. 🕶 This is no real suprise, but hopefully for dashboards like this can really expand the understrand of issues in our city! 🌃 Thanks for staying. Cookie Emoji for you 🍪.  